In [31]:
import torch 
import numpy as np
import ipytest
ipytest.autoconfig()

In [32]:
def Conv2D(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros'):
    def generate_bias():
        return torch.rand(out_channels) if bias else torch.zeros(out_channels)

    def apply_padding(matrix):
        if padding_mode == 'zeros':
            pad = torch.nn.ZeroPad2d(padding)
        elif padding_mode == 'reflect':
            pad = torch.nn.ReflectionPad2d(padding)
        elif padding_mode == 'replicate':
            pad = torch.nn.ReplicationPad2d(2)
        elif padding_mode == 'circular':
            pad = torch.nn.CircularPad2d(padding)

        return pad(matrix)

    def generate_filter():
        if isinstance(kernel_size, tuple):
            return torch.rand(out_channels, in_channels // groups, *kernel_size)
        elif isinstance(kernel_size, int):
            return torch.rand(out_channels, in_channels // groups, kernel_size, kernel_size)

    def perform_convolution(matrix, filter):
        result = []
        for l in range(out_channels):
            feature_map = np.array([])

            for i in range(0, matrix.shape[1] - ((filter.shape[2] - 1) * dilation + 1) + 1, stride):
                for j in range(0, matrix.shape[2] - ((filter.shape[3] - 1) * dilation + 1) + 1, stride):
                    total = calculate_convolution_sum(matrix, filter, l, i, j)
                    feature_map = np.append(feature_map, float(total + bias_values[l]))

            result.append(feature_map.reshape(
                (matrix.shape[1] - ((filter.shape[2] - 1) * dilation + 1)) // stride + 1,
                (matrix.shape[2] - ((filter.shape[3] - 1) * dilation + 1)) // stride + 1
            ))

        return np.array(result), torch.tensor(np.array(filter)), torch.tensor(np.array(bias_values))

    def calculate_convolution_sum(matrix, filter, l, i, j):
        total = 0
        for c in range(in_channels // groups):
            val = select_input_slice(matrix, l, c, i, j)
            local_sum = (val * filter[l][c]).sum()
            total += local_sum
        return total

    def select_input_slice(matrix, l, c, i, j):
        if groups > 1:
            return matrix[l * (in_channels // groups) + c][i:i + (filter.shape[2] - 1) * dilation + 1:dilation,
                                                          j:j + (filter.shape[3] - 1) * dilation + 1:dilation]
        else:
            return matrix[c][i:i + (filter.shape[2] - 1) * dilation + 1:dilation,
                             j:j + (filter.shape[3] - 1) * dilation + 1:dilation]

    bias_values = generate_bias()
    filter = generate_filter()

    def convolution(matrix):
        matrix = apply_padding(matrix)
        result, filter_result, bias_result = perform_convolution(matrix, filter)
        return result, filter_result, bias_result

    return convolution

In [33]:
def create_random_tensor(shape):
    return torch.rand(shape)

def compare_convolutions(custom_conv, torch_conv, input_tensor):
    result, kernel, bias_val = custom_conv(input_tensor)
    torch_conv.weight.data = torch.tensor(kernel)
    torch_conv.bias.data = torch.tensor(bias_val)

    custom_result = str(np.round(result, 2))
    torch_result = str(np.round(np.array(torch_conv(input_tensor).data), 2))

    assert torch_result == custom_result

In [34]:
def test_1():
    tensor = create_random_tensor((10, 28, 28))
    conv = Conv2D(               in_channels=10, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
    torch_conv = torch.nn.Conv2d(in_channels=10, out_channels=2, kernel_size=3, stride=1, padding=0, dilation=1, groups=2, bias=True, padding_mode='zeros')
    compare_convolutions(conv, torch_conv, tensor)

def test_2():
    tensor = create_random_tensor((3, 100, 100))
    conv = Conv2D(               in_channels=3, out_channels=1, kernel_size=4, stride=2, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
    torch_conv = torch.nn.Conv2d(in_channels=3, out_channels=1, kernel_size=4, stride=2, padding=0, dilation=2, groups=1, bias=True, padding_mode='zeros')
    compare_convolutions(conv, torch_conv, tensor)

def test_3():
    tensor = create_random_tensor((4, 4, 4))
    conv = Conv2D(               in_channels=4, out_channels=4, kernel_size=1, stride=1, padding=0, dilation=1, groups=4, bias=True, padding_mode='zeros')
    torch_conv = torch.nn.Conv2d(in_channels=4, out_channels=4, kernel_size=1, stride=1, padding=0, dilation=1, groups=4, bias=True, padding_mode='zeros')
    compare_convolutions(conv, torch_conv, tensor)
    
def test_4():
    tensor = create_random_tensor((3, 128, 128))
    conv = Conv2D(               in_channels=3, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
    torch_conv = torch.nn.Conv2d(in_channels=3, out_channels=1, kernel_size=1, stride=1, padding=0, dilation=1, groups=1, bias=True, padding_mode='zeros')
    compare_convolutions(conv, torch_conv, tensor)

In [35]:
ipytest.run()

....                                                                                         [100%]
======================================== warnings summary =========================================
t_ba46173c862a48ccbf13ec9840edf6e3.py::test_1
t_ba46173c862a48ccbf13ec9840edf6e3.py::test_2
t_ba46173c862a48ccbf13ec9840edf6e3.py::test_3
t_ba46173c862a48ccbf13ec9840edf6e3.py::test_4
  C:\Users\Никола\AppData\Local\Temp\ipykernel_14888\69101569.py:6: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
    torch_conv.weight.data = torch.tensor(kernel)

t_ba46173c862a48ccbf13ec9840edf6e3.py::test_1
t_ba46173c862a48ccbf13ec9840edf6e3.py::test_2
t_ba46173c862a48ccbf13ec9840edf6e3.py::test_3
t_ba46173c862a48ccbf13ec9840edf6e3.py::test_4
  C:\Users\Никола\AppData\Local\Temp\ipykernel_14888\69101569.py:7: UserWarning: To copy construct from a tensor, it 

<ExitCode.OK: 0>